*version 190910.0*

# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment.

In addition, I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

## Notes

1. Partial marks may be awarded if the answer is incorrect, or removed even if the answer is correct! Within each function show your understanding of course concepts such as regex, `groupby()`, `apply()`, etc. as appropriate to the question. If you unnecessarily hand code data in your functions instead of using pandas cleaning mechanisms you may be docked grades.
2. Do not including data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
3. I highly suggest that you first tackle the correlation questions, as they are all similar and worth the majority of grades for this assignment. This is by design!
4. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
5. This assignment **is not autograded**. Go to the course shell and upload your .ipynb file there as assignment4.ipynb.
6. You can earn up to 100% on this assignment by creating correlations for a single year (2016) for which there is population data. You can earn up to 105% on this assignment by going further, and creating correlations for multiple years (e.g. across the years of data we have on the sports), but it's expected that there would be a short discussion of the limitations of your approach as far as interpretation goes. Finally, if you find new population data that covers a broader set of years and integrate it into a solution across these years and sports and provide that interpretation you can earn up to 110% on the assignment.

In [65]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import itertools
import re

## Initialization of Data: Read and Group Sport Stats

__Procedure__: first I load sport stats data and do some basic clean up and manipulation.

Importantly for NHL data: 
- I correct half of data attributed to 2017 to be from 2016. 
- I calculate the W/L% ratio as W/(W+L) from the data at hand. 

In [99]:
mlb_df = pd.read_csv("assets/mlb.csv")

nhl_df=pd.read_csv("assets/nhl.csv")
nhl_df.loc[nhl_df.duplicated(subset = ['team', 'year']), 'year'] = 2016 # correct second ocurrence of 2017 data to be of 2016
nhl_df['W-L%'] = pd.to_numeric(nhl_df['W'], errors = 'coerce')/ (pd.to_numeric(nhl_df['W'], errors = 'coerce') + pd.to_numeric(nhl_df['L'], errors = 'coerce')) # create w/l% as defined by W/(W+L)

nba_df=pd.read_csv("assets/nba.csv")
nba_df = nba_df.rename({'W/L%':'W-L%'}, axis = 1) # basic renaming

nfl_df=pd.read_csv("assets/nfl.csv")

sport_stats = {'MLB': mlb_df, 'NHL': nhl_df, 'NBA':nba_df, 'NFL': nfl_df} # grouping for clarity

## Manipulation of Metropolitan Data

__Procedure Explanation__

- First I loaded the raw html to a string, and performed two clean up operations:
    1. replace hyperlinked text with the title tag in the href code, as it contains an already cleaned sports team name. An important feature during this cleanup is that I encircled contiguous team names in double quotes, so as to easily separate them afterwards when doing regex. 
    2. removed the notes that appeared next to the team names
- This worked html was then fed to panda's html parser to extract the relevant cities table. 
- Then I expanded into several rows teams that were part of the same area, so that they could be merged properly later on. 
- I performed some additional cleanup for NaNs and legibility.
- Finally I hard coded some team names that were not present in the table, but were available on the individual sport files. 


In [120]:
# pre-load HTML to clean team names in raw HTML
cities = open("assets/wikipedia_data.html", "r").read()

pat_title = re.compile(r'(<a.*?title=\"(?P<title>.*?)\".*?<\/a>)') 
cities = pat_title.sub('\"\g<title>\"', cities) # replace bad content team names, with title tag names from href link
pat_notes = re.compile(r'<sup(?:.|\n)*?<\/sup>') 
cities = pat_notes.sub('', cities) # remove notes from names

# load DF from cleaned HTML
cities = pd.read_html(cities)[1]
cities = cities.iloc[:-1,[0,3,5,6,7,8]]

# Rename columns for clarity
cities = cities.rename({'"Metropolitan area"': 'Area', 'Population (2016 est.)': 'Population'}, axis = 1)

# Clean NaNs 
cities = cities.replace('—',np.nan)

# Expand Areas with Multiple Teams
cities = (cities.set_index(['Area', 'Population'])
   .stack()
   .str.strip('"') 
   .str.split('""', expand=True) # splitting contiguous team names
   .stack()
   .unstack(-2)
   .reset_index(-1, drop=True)
   .reset_index()
   .fillna('')
)

# Clean Team and Area Names
cities['Area'] = cities['Area'].str.strip('"')
cities = (cities
    .replace(to_replace = r'(?P<team>^[A-Z1-9][\w\. \-]+)(.*)', value = '\g<team>', regex = True) # further clean up of Team names 
    .replace(to_replace = r' metropolitan area', value = '', regex = True) # remove 'metropolitan area' from area names
    .apply(lambda x: x.str.strip()) # further cleaning of whitespace 
)

# Change/Coerce to Relevant dtype
cities['Population'] = pd.to_numeric(cities.Population)

# Manually Add San Diego Chargers, St. Louis Rams, Los Angeles Angels of Anneheim, Charlotte Bobcats and Phoenix Coyotes as they don't Exist in the DB due to Relocating/Renaming
area = 'San Diego County'
aux = pd.Series({'Area': area, 'Population': cities[cities.Area == area].Population.mean(), 'NFL':'San Diego Chargers', 'MLB':'', 'NBA':'', 'NHL':''}, name = 'Added')
cities = cities.append(aux)
area = 'Greater St. Louis'
aux = pd.Series({'Area': area, 'Population': cities[cities.Area == area].Population.mean(), 'NFL':'St. Louis Rams', 'MLB':'', 'NBA':'', 'NHL':''}, name = 'Added')
cities = cities.append(aux)
area = 'Charlotte'
aux = pd.Series({'Area': area, 'Population': cities[cities.Area == area].Population.mean(), 'NFL':'', 'MLB':'', 'NBA':'Charlotte Bobcats', 'NHL':''}, name = 'Added')
cities = cities.append(aux)
area = 'Los Angeles'
aux = pd.Series({'Area': area, 'Population': cities[cities.Area == area].Population.mean(), 'NFL':'', 'MLB':'Los Angeles Angels of Anaheim', 'NBA':'', 'NHL':''}, name = 'Added')
cities = cities.append(aux)
area = 'Phoenix'
aux = pd.Series({'Area': area, 'Population': cities[cities.Area == area].Population.mean(), 'NFL':'', 'MLB':'', 'NBA':'', 'NHL':'Phoenix Coyotes'}, name = 'Added')
cities = cities.append(aux)

## Process and Merge Sports Data with Population Data

The following two functions are used to clean up the sports data and to merge it with the population data. 

1. clean_stats Function:
    - It cleans up team names by doing a regex and replacing what it founds with the contents of a group which I called team. 
    - It also coerces W-L% to numerical data, and removes any rows that produce a NaN during coercion, this is to eliminate rows that do not have relevant team data, like headers. 
2. merge_panels Function:
    - first it cleans up the sports panel with the previous function
    - then subselects just relevant columns and rows for a given year
    - merges resulting data with population data while at the same time aggregating results with an average as indicated at the area level. 

In [121]:
def clean_stats(sport_stats):
    sport_stats['team'] = sport_stats['team'].replace(to_replace = r'(?P<team>^[A-Z1-9][\w\. \-]+)(.*)', value = '\g<team>', regex = True) # clean team names
    sport_stats['W-L%'] = pd.to_numeric(sport_stats['W-L%'], errors = 'coerce') 
    sport_stats = sport_stats.dropna(subset = ['W-L%']) # remove non-relevant non-numeric rows
    return sport_stats

In [122]:
def merge_panels(sport_stats, cities, sport, year):
    # clean panel
    sport_stats = clean_stats(sport_stats[sport])
    
    # select relevant data
    sport_stats = sport_stats[sport_stats.year == year][['team', 'W-L%']]
    cities = cities[['Area', 'Population', sport]].rename({sport:'team'}, axis = 1)
    
    # merge and group relevant data
    sport_stats = sport_stats.merge(cities, on = 'team', how = 'left') # merge with population
    sport_stats = sport_stats.groupby('Area').agg({'Population': np.mean, 'W-L%': np.mean})
    
    return sport_stats

### Calculation Functions

Below are the functions used to calculate main results. The most important ones are two:

1. sport_correlation Function:
    - given a sport, panel of sport stats, panel of cities with population/teams and a year it calculates the correlation of W-L% with Population
    - it mainly uses the merge_panels function described earlier
    - it includes an error grabbing routine in case there are no relevant data to be used (wrong year for example)
    
    
2. sports_team_performance Function:
    - given a panel of sport stats, panel of cities with population/teams and a year it calculates the paired t-test for all combinations of sports in the sports stats panel 
    - it uses the merge_panel clean up function described earlier, and importantly it merges both frames with an inner join to only consider areas that have sport teams in both sports 


3. specific sports Functions: 
    - those are just calls to the more general function

In [123]:
def sport_correlation(sport, sport_stats, cities, year = 2017):
    data = merge_panels(sport_stats, cities, sport, year)
    population_by_region = data['Population']
    win_loss_by_region = data['W-L%']
    try: 
        value = stats.pearsonr(population_by_region, win_loss_by_region)
    except:
        value = (np.nan, np.nan)
        
    return value

def nhl_correlation(year = 2016): # worth 20% of the assignment grade
    return sport_correlation('NHL', sport_stats, cities, year)

def nba_correlation(year = 2016): # worth 20% of the assignment grade
    return sport_correlation('NBA', sport_stats, cities, year)

def mlb_correlation(year = 2016): # worth 20% of the assignment grade
    return sport_correlation('MLB', sport_stats, cities, year)

def nfl_correlation(year = 2016): # worth 20% of the assignment grade
    return sport_correlation('NFL', sport_stats, cities, year)

def sports_team_performance(sport_stats, cities, year = 2016) : # worth 20% of the assignment grade
    sports = sport_stats.keys()
    p_values = pd.DataFrame({k:np.nan for (k) in sports},index=sports)
    
    for pair in itertools.permutations(sports, 2):
        sport1 = merge_panels(sport_stats, cities, pair[0], year)[['W-L%']]
        sport2 = merge_panels(sport_stats, cities, pair[1], year)[['W-L%']]
        both_sports = sport1.merge(sport2, on = 'Area', how = 'inner')
        p_values.loc[pair[0], pair[1]] = stats.ttest_rel(both_sports['W-L%_x'], both_sports['W-L%_y']).pvalue
    return p_values

## Main Results

### Parts 1 - 4: Correlations

In [124]:
for sport in sport_stats.keys():
    print('The correlation for 2016 of population and W-L% for {} is {}'.format(sport, sport_correlation(sport, sport_stats, cities, year = 2016)[0]))

The correlation for 2016 of population and W-L% for MLB is 0.2461879894080591
The correlation for 2016 of population and W-L% for NHL is 0.31580627948704787
The correlation for 2016 of population and W-L% for NBA is -0.18769238444022995
The correlation for 2016 of population and W-L% for NFL is -0.06106574171738971


### Part 5: Paired Tests

__Interpretation__

As its noticeable from the results below the null hypothesis of equal performance of teams for differents sports given an area can be rejected at a 5% confidence level for the pair __MLB/NHL__ and __NHL/NFL__.

In [126]:
sports_team_performance(sport_stats, cities, year = 2016)

,MLB,NHL,NBA,NFL
MLB,NaN,0.000111,0.799103,0.572215
NHL,0.000111,NaN,0.074380,0.039501
NBA,0.799103,0.074380,NaN,0.950015
NFL,0.572215,0.039501,0.950015,NaN


In [127]:
sports_team_performance(sport_stats, cities, year = 2016) < 0.05

,MLB,NHL,NBA,NFL
MLB,False,True,False,False
NHL,True,False,False,True
NBA,False,False,False,False
NFL,False,True,False,False


## Optional Results

### Part 3: Correlation over the Years

__Limitation of the exercise__: the correlation over the years quite simply uses static population data for each area, which of course is going to affect correlation coefficients for years different to 2016, given this is the one for which we know the population figures. Any changes observed over time in the coefficients might be due to changing demographics not accounted for, and not due to the relationship per se. 

__Procedure__: the function below quite simply calls the sport_correlation function as defined for the main portion of the assignment, but changes the years for which it is calculated. Importantly, this does not change the cities frame that contains population data, but just filters sport stats based on the year.

In [128]:
def yearly_correlation(sport_stats, cities):
    sports = sport_stats.keys()
    correlations = pd.DataFrame()
    
    for year in range(2014,2019):
        for sport in sports:
            correlations.loc[year, sport] = sport_correlation(sport, sport_stats, cities, year)[0]
        
    return correlations

### Table of Correlation Coefficient for Sports with Static Area Population over Time

In [129]:
yearly_correlation(sport_stats, cities)

,MLB,NHL,NBA,NFL
2014,0.031091,0.229168,0.109021,-0.190503
2015,0.206464,0.160268,-0.065090,-0.077871
2016,0.246188,0.315806,-0.187692,-0.061066
2017,0.063575,0.225120,-0.200451,-0.189007
2018,0.150037,0.012486,-0.176364,0.004282


### Part 4: Additional Population Data Set

For this part of the assignment I downloaded metropolitan population data from the OCDE website (available here, https://stats.oecd.org/Index.aspx?DataSetCode=CITIES#).

__Assumptions:__
   - I ignored in the analysis those areas for which the OCDE has no population data, but for which the original Wikipedia article has data. 
   - The OCDE data is updated until 2016, so I extrapolated the population linearly for each area on a yearly basis from 2017 to 2018
   
__Explanation of the procedure:__
- First I load the new population data and extrapolate data for unavailable years. 
- Then I constructed a function, city_population(cities, year), which given the original 'cities' frame and a year, updates population figures for the 'cities' table. 
    - The linkage of areas is made using an externally provided area-names dictionary that I constructed by hand. 
- The next step was to construct a slightly modified version of the function that gives the correlation for different years for each sport that updates the 'cities' frame before handing it over to the function that computes the correlation.
    - The previous functions were constructed in such a way that the 'cities' frame was always to be given, so it was easy to just provide a new 'cities' frame.

__Limitations:__
- One important limitation is that the metropolitan areas might not necesarrily be the same as those on the original table, I just randomly checked a couple of cities to test whether the populations were more or less of the same order of magnitude
- Another limitation is that some areas that have important teams were ignored due to missing data (e.g. Cleveland)
- One last limitation is that the extrapolation procedure might not reflect properly correct changes in demographics over time


In [130]:
population_data = pd.read_csv('cities.csv')
population_data = population_data[population_data.Variables == 'Population, All ages. Administrative data']#[['Metropolitan areas', 'Year', 'Value ']]
population_data = population_data.drop(['METRO_ID', 'Variables'], axis = 1)
population_data.columns = ['Area', 'Year', 'Population']

extrapolated = pd.pivot_table(population_data, index = 'Area', values = 'Population', columns = 'Year')
extrapolated[2017] = extrapolated[2016]*(1+(extrapolated[2016]/extrapolated[2015] - 1))
extrapolated[2018] = extrapolated[2017]*(1+(extrapolated[2017]/extrapolated[2016] - 1))
population_data = extrapolated.unstack().reset_index()    
population_data.columns = ['Year', 'Area', 'Population']

names_dictionary = pd.read_csv('areas.csv')

In [131]:
def city_population(cities, year):
    cities = cities.merge(names_dictionary, on = 'Area', how = 'left').merge(population_data[population_data.Year == year], left_on = 'SecondArea', right_on = 'Area', how = 'left')
    cities = cities.drop(['Population_x', 'Area_y', 'Year', 'SecondArea'], axis = 1)
    cities = cities.rename({'Area_x': 'Area', 'Population_y': 'Population'}, axis = 1)
    cities = cities.dropna(subset = ['Population'])
    return cities

In [132]:
def yearly_correlation_changing_pop(sport_stats, cities):
    sports = sport_stats.keys()
    correlations = pd.DataFrame()
    
    for year in range(2014,2019):
        cities = city_population(cities, year)
        for sport in sports:
            correlations.loc[year, sport] = sport_correlation(sport, sport_stats, cities, year)[0]
        
    return correlations

### Table of Correlation Coefficient for Sports with Changing Area Population over Time

The result below corresponds to the correlation coefficient of W/L% with population with changing population over time. This overcomes the limitation on the previous section for which there might be a factor unaccounted for when calculating correlations. 

__The results do not change significantly.__ Meaning that apart from the result for NHL2018 none of the correlations changes sign and were of approximate magnitude. Its important to note that population measures do not coincide even for the year 2016, as such, I did not expect to get the exact same values. 

__However I did expect to see even fewer magnitudal changes to the correlations__, as I thought that the variance in W/L% would completely overwhelm changing population metrics as those tend to be quite stable even for different datasets. 

In [133]:
yearly_correlation_changing_pop(sport_stats, cities)

,MLB,NHL,NBA,NFL
2014,0.256992,0.131513,0.053224,-0.113911
2015,0.287228,0.032083,-0.075039,-0.081683
2016,0.393244,0.314759,-0.179691,-0.159941
2017,0.207847,0.204622,-0.183235,-0.261311
2018,0.033637,-0.074248,-0.169876,0.059023


In [36]:
!cowsay 'hopefully I get full grades!'

 ______________________________
< hopefully I get full grades! >
 ------------------------------
        \   ^__^
         \  (oo)\_______
            (__)\       )\/\
                ||----w |
                ||     ||
